# 第11课：时间序列预测

## 学习目标
- 理解时间序列数据的特点
- 掌握时间序列分解
- 学习 ARIMA 模型
- 使用机器学习进行时间序列预测

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

## 1. 时间序列基础

时间序列是按时间顺序排列的数据点序列。

**特点**：
- 数据有时间依赖性
- 可能存在趋势、季节性、周期性
- 常见应用：股票预测、销量预测、天气预报

In [ ]:
# 创建模拟时间序列数据
np.random.seed(42)

# 生成日期
dates = pd.date_range(start='2020-01-01', periods=365*3, freq='D')

# 生成数据：趋势 + 季节性 + 噪声
trend = np.linspace(100, 200, len(dates))  # 上升趋势
seasonal = 20 * np.sin(2 * np.pi * np.arange(len(dates)) / 365)  # 年度季节性
weekly = 5 * np.sin(2 * np.pi * np.arange(len(dates)) / 7)  # 周季节性
noise = np.random.randn(len(dates)) * 5

values = trend + seasonal + weekly + noise

# 创建 DataFrame
ts_data = pd.DataFrame({'date': dates, 'value': values})
ts_data.set_index('date', inplace=True)

print(ts_data.head())
print(f"\n数据范围: {ts_data.index.min()} 到 {ts_data.index.max()}")

In [ ]:
# 可视化
plt.figure(figsize=(14, 5))
plt.plot(ts_data.index, ts_data['value'], linewidth=0.8)
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Time Series Data')
plt.grid(True, alpha=0.3)
plt.show()

## 2. 时间序列分解

将时间序列分解为：
- **趋势 (Trend)**：长期变化方向
- **季节性 (Seasonal)**：固定周期的重复模式
- **残差 (Residual)**：剩余的随机波动

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# 加法分解
decomposition = seasonal_decompose(ts_data['value'], model='additive', period=365)

fig, axes = plt.subplots(4, 1, figsize=(14, 10))

decomposition.observed.plot(ax=axes[0], title='Original')
decomposition.trend.plot(ax=axes[1], title='Trend')
decomposition.seasonal.plot(ax=axes[2], title='Seasonal')
decomposition.resid.plot(ax=axes[3], title='Residual')

plt.tight_layout()
plt.show()

## 3. 平稳性检验

很多时间序列模型要求数据是平稳的（均值和方差不随时间变化）

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(series):
    """ADF 单位根检验"""
    result = adfuller(series.dropna())
    print('ADF 统计量:', result[0])
    print('p-value:', result[1])
    print('临界值:')
    for key, value in result[4].items():
        print(f'  {key}: {value}')
    
    if result[1] < 0.05:
        print('\n结论: 数据是平稳的 (拒绝原假设)')
    else:
        print('\n结论: 数据不是平稳的 (无法拒绝原假设)')

print("原始数据平稳性检验:")
adf_test(ts_data['value'])

In [ ]:
# 差分使数据平稳
ts_diff = ts_data['value'].diff().dropna()

print("一阶差分后平稳性检验:")
adf_test(ts_diff)

In [ ]:
# 可视化差分结果
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(ts_data['value'])
axes[0].set_title('Original Data')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ts_diff)
axes[1].set_title('First Difference')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. ACF 和 PACF

- **ACF (自相关函数)**：当前值与滞后值的相关性
- **PACF (偏自相关函数)**：排除中间滞后影响后的相关性

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# 原始数据
plot_acf(ts_data['value'], ax=axes[0, 0], lags=50, title='ACF - Original')
plot_pacf(ts_data['value'], ax=axes[0, 1], lags=50, title='PACF - Original')

# 差分后
plot_acf(ts_diff, ax=axes[1, 0], lags=50, title='ACF - Differenced')
plot_pacf(ts_diff, ax=axes[1, 1], lags=50, title='PACF - Differenced')

plt.tight_layout()
plt.show()

## 5. ARIMA 模型

ARIMA (AutoRegressive Integrated Moving Average) 是经典的时间序列模型。

参数 (p, d, q)：
- **p**: 自回归阶数
- **d**: 差分阶数
- **q**: 移动平均阶数

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# 划分训练集和测试集
train_size = int(len(ts_data) * 0.8)
train, test = ts_data[:train_size], ts_data[train_size:]

print(f"训练集: {len(train)} 天")
print(f"测试集: {len(test)} 天")

In [ ]:
# 训练 ARIMA 模型
model = ARIMA(train['value'], order=(2, 1, 2))
model_fit = model.fit()

print(model_fit.summary())

In [ ]:
# 预测
predictions = model_fit.forecast(steps=len(test))

# 评估
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(test['value'], predictions))
mae = mean_absolute_error(test['value'], predictions)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

In [ ]:
# 可视化预测结果
plt.figure(figsize=(14, 6))

plt.plot(train.index, train['value'], label='Training', color='blue')
plt.plot(test.index, test['value'], label='Actual', color='green')
plt.plot(test.index, predictions, label='Predicted', color='red', linestyle='--')

plt.xlabel('Date')
plt.ylabel('Value')
plt.title('ARIMA Forecast')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. 自动 ARIMA (Auto ARIMA)

In [ ]:
# 安装: pip install pmdarima
try:
    import pmdarima as pm
    PMDARIMA_AVAILABLE = True
    print("pmdarima 已安装")
except ImportError:
    PMDARIMA_AVAILABLE = False
    print("请先安装 pmdarima: pip install pmdarima")

In [ ]:
if PMDARIMA_AVAILABLE:
    import pmdarima as pm

    # 自动选择最佳参数
    auto_arima = pm.auto_arima(
        train['value'],
        start_p=0, start_q=0,
        max_p=5, max_q=5,
        d=None,  # 自动确定差分阶数
        seasonal=False,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )

    print(f"\n最佳模型: {auto_arima.order}")
else:
    print("跳过 Auto ARIMA（未安装 pmdarima）")

In [ ]:
if PMDARIMA_AVAILABLE:
    # 使用自动选择的模型预测
    auto_predictions = auto_arima.predict(n_periods=len(test))

    rmse_auto = np.sqrt(mean_squared_error(test['value'], auto_predictions))
    print(f"Auto ARIMA RMSE: {rmse_auto:.4f}")
else:
    rmse_auto = None
    print("跳过 Auto ARIMA 预测（未安装 pmdarima）")

## 7. 机器学习方法

将时间序列问题转换为监督学习问题

In [ ]:
def create_features(df, lag_days=7):
    """创建滞后特征"""
    df = df.copy()
    
    # 滞后特征
    for i in range(1, lag_days + 1):
        df[f'lag_{i}'] = df['value'].shift(i)
    
    # 滚动统计特征
    df['rolling_mean_7'] = df['value'].shift(1).rolling(window=7).mean()
    df['rolling_std_7'] = df['value'].shift(1).rolling(window=7).std()
    df['rolling_mean_30'] = df['value'].shift(1).rolling(window=30).mean()
    
    # 时间特征
    df['day_of_week'] = df.index.dayofweek
    df['day_of_month'] = df.index.day
    df['month'] = df.index.month
    df['day_of_year'] = df.index.dayofyear
    
    return df.dropna()

# 创建特征
ts_features = create_features(ts_data, lag_days=7)
print(ts_features.head())

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge

# 准备数据
feature_cols = [col for col in ts_features.columns if col != 'value']
X = ts_features[feature_cols]
y = ts_features['value']

# 时间序列分割（不能随机分割）
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"训练集: {X_train.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
# 训练多个模型
models = {
    'Ridge': Ridge(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    results.append({'Model': name, 'RMSE': rmse, 'MAE': mae})
    print(f"{name}: RMSE={rmse:.4f}, MAE={mae:.4f}")

results_df = pd.DataFrame(results)

In [ ]:
# 使用最佳模型进行可视化
best_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)

plt.figure(figsize=(14, 6))

plt.plot(y_train.index, y_train, label='Training', color='blue', alpha=0.7)
plt.plot(y_test.index, y_test, label='Actual', color='green')
plt.plot(y_test.index, y_pred_best, label='Predicted (GB)', color='red', linestyle='--')

plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Gradient Boosting Time Series Forecast')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# 特征重要性
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'][:15], feature_importance['importance'][:15])
plt.xlabel('Importance')
plt.title('Feature Importance for Time Series Prediction')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 8. Prophet 预测

Facebook Prophet 是一个强大的时间序列预测库

In [ ]:
# 安装: pip install prophet
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
    print("Prophet 已安装")
except ImportError:
    PROPHET_AVAILABLE = False
    print("请先安装 prophet: pip install prophet")

In [ ]:
if PROPHET_AVAILABLE:
    from prophet import Prophet

    # 准备 Prophet 格式的数据
    prophet_df = ts_data.reset_index()
    prophet_df.columns = ['ds', 'y']

    # 分割数据
    prophet_train = prophet_df[:train_size]
    prophet_test = prophet_df[train_size:]

    # 训练模型
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False
    )
    prophet_model.fit(prophet_train)
else:
    print("跳过 Prophet 训练（未安装）")

In [ ]:
if PROPHET_AVAILABLE:
    # 预测
    future = prophet_model.make_future_dataframe(periods=len(prophet_test))
    forecast = prophet_model.predict(future)

    # 评估
    prophet_predictions = forecast['yhat'].iloc[-len(prophet_test):].values
    prophet_rmse = np.sqrt(mean_squared_error(prophet_test['y'], prophet_predictions))
    print(f"Prophet RMSE: {prophet_rmse:.4f}")
else:
    prophet_rmse = None
    print("跳过 Prophet 预测（未安装）")

In [ ]:
if PROPHET_AVAILABLE:
    # Prophet 可视化
    fig = prophet_model.plot(forecast)
    plt.title('Prophet Forecast')
    plt.show()
else:
    print("跳过 Prophet 可视化（未安装）")

In [ ]:
if PROPHET_AVAILABLE:
    # 分解图
    fig = prophet_model.plot_components(forecast)
    plt.show()
else:
    print("跳过 Prophet 分解图（未安装）")

## 9. 模型对比

In [ ]:
# 汇总所有模型结果
all_results = [
    {'Model': 'ARIMA', 'RMSE': rmse},
    {'Model': 'Ridge', 'RMSE': results_df[results_df['Model']=='Ridge']['RMSE'].values[0]},
    {'Model': 'Random Forest', 'RMSE': results_df[results_df['Model']=='Random Forest']['RMSE'].values[0]},
    {'Model': 'Gradient Boosting', 'RMSE': results_df[results_df['Model']=='Gradient Boosting']['RMSE'].values[0]},
]

# 添加可选模型结果
if PMDARIMA_AVAILABLE and rmse_auto is not None:
    all_results.append({'Model': 'Auto ARIMA', 'RMSE': rmse_auto})
if PROPHET_AVAILABLE and prophet_rmse is not None:
    all_results.append({'Model': 'Prophet', 'RMSE': prophet_rmse})

comparison_df = pd.DataFrame(all_results).sort_values('RMSE')
print("模型对比:")
print(comparison_df.to_string(index=False))

In [ ]:
# 可视化对比
plt.figure(figsize=(10, 6))
plt.barh(comparison_df['Model'], comparison_df['RMSE'], color='steelblue')
plt.xlabel('RMSE')
plt.title('Model Comparison')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 10. 练习题

### 练习1：股票数据预测
使用真实的股票数据进行预测

In [ ]:
# 提示: 可以使用 yfinance 库获取股票数据
# pip install yfinance
# import yfinance as yf
# data = yf.download('AAPL', start='2020-01-01', end='2023-12-31')

# 在这里编写代码


### 练习2：多步预测
预测未来 30 天的数据

In [ ]:
# 在这里编写代码


## 11. 本课小结

### 时间序列方法对比

| 方法 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| ARIMA | 经典方法，可解释 | 需要平稳性，难以捕捉复杂模式 | 短期预测 |
| Prophet | 自动处理季节性，易用 | 可能不适合复杂数据 | 有明显季节性的数据 |
| ML方法 | 能捕捉复杂模式 | 需要特征工程 | 有外部特征的预测 |

### 关键步骤

1. **数据探索**：可视化、分解、检查平稳性
2. **特征工程**：滞后特征、滚动统计、时间特征
3. **模型选择**：根据数据特点选择合适模型
4. **评估**：使用 RMSE、MAE 等指标
5. **预测**：注意不要使用未来信息

### 注意事项

- 时间序列不能随机分割，必须按时间顺序
- 注意数据泄露：特征不能使用未来信息
- 考虑季节性和趋势的影响